In [ ]:
import shutil
import os

# _unsloth_temporary_saved_buffers
# huggingface_tokenizers_cache
# outputs
# unsloth_compiled_cache

folder_path = "/content/_unsloth_temporary_saved_buffers"  # Replace with your folder's name

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder does not exist: {folder_path}")

folder_path = "/content/huggingface_tokenizers_cache"  # Replace with your folder's name

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder does not exist: {folder_path}")

folder_path = "/content/outputs"  # Replace with your folder's name

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder does not exist: {folder_path}")

folder_path = "/content/unsloth_compiled_cache"  # Replace with your folder's name

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder does not exist: {folder_path}")

Folder does not exist: /content/_unsloth_temporary_saved_buffers
Folder does not exist: /content/huggingface_tokenizers_cache
Folder does not exist: /content/outputs
Folder does not exist: /content/unsloth_compiled_cache


In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj", "embed_tokens",
                      "dense_h_to_4h", "dense_4h_to_h"], #embed_tokens / lm_head
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3417,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!
Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!
Unsloth: Offloading input_embeddings to disk to save VRAM


Unsloth 2025.8.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/Phi-3.5-Mini-Instruct")
print("🧠 Max context length:", tokenizer.model_max_length)

print("Model config context length:", model.config.max_position_embeddings)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

🧠 Max context length: 131072
Model config context length: 131072


In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3.5",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/all_datasets.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/46650 [00:00<?, ? examples/s]

In [6]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/46650 [00:00<?, ? examples/s]

Map:   0%|          | 0/46650 [00:00<?, ? examples/s]

In [7]:
dataset[15]["conversations"]

[{'role': 'user',
  'content': 'Classify this email as spam or ham:\n\n"Quoting Brendan Kehoe (brendan@zen.org):\\n\\n> As a workaround, the various distributions could use a GPG singature\\n> to verify correctness of the file.  Since the distributor\'s secret key\\n> is required to create that signature, it would add a pretty\\n> significant step that would have to be taken...'},
 {'role': 'assistant',
  'content': 'spam — because the content matches patterns typical of spam emails.'}]

In [8]:
dataset[51]["text"]

'<|user|>\nClassify this email as spam or ham:\n\n"Al white wrote:\\n\\n >erm... it runs Solaris x86 as standard...\\n\\nIt runs Solaris 8 x86 as standard.\\n(I was joking Al)\\n\\nM.\\n\\n\\n-- \\nIrish Linux Users\' Group: ilug@linux.ie\\nhttp://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.\\nList maintainer: listmaster@linux.ie\\n\\n"...<|end|>\n<|assistant|>\nspam — because the content matches patterns typical of spam emails.<|end|>\n'

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Trainer with evaluation
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        warmup_steps = 100,
        # max_steps = 60,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", #
        seed = 3417,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/46650 [00:00<?, ? examples/s]

In [10]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|user|>\n",
    response_part = "<|assistant|>\n",
)

Map (num_proc=2):   0%|          | 0/46650 [00:00<?, ? examples/s]

In [11]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|user|> Classify this email as spam or ham:\n\n"Hi All,\\nI\'m trying to set up the following:\\n\\n1. A Linux server running with a modem for internet connectivity and an\\nethernet card for LAN connectivity\\n2. Other LAN pcs with ethernet cards, using the Linux server for\\nDNS/DHCP etc.\\n\\nBasically, I want to route any non LAN traffic through the ...<|end|><|assistant|> spam — because the content matches patterns typical of spam emails.<|end|>'

In [12]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                        spam — because the content matches patterns typical of spam emails.<|end|>'

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 46,650 | Num Epochs = 1 | Total steps = 11,663
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 218,038,272 of 4,039,117,824 (5.40% trained)


Step,Training Loss,entropy
1,8.099500,0
2,8.170700,No Log
3,7.785700,No Log
4,8.049900,No Log
5,6.958400,No Log
6,7.176400,No Log
7,6.967300,No Log
8,5.784900,No Log
9,5.685700,No Log
10,4.831600,No Log


Unsloth: Will smartly offload gradients to save VRAM!


In [14]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3.5",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Classify this email as spam or ham:\n\nGet rich quick, click here!"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|user|> Classify this email as spam or ham:\n\nGet rich quick, click here!<|end|><|assistant|> ham — because the content matches patterns typical of ham emails.<|end|> 1 — because the content matches typical patterns of ham emails.<|end|> 1 — because the content matches typical of ham emails.<|end|> 1 — because the content matches typical patterns of 1 emails.<|end|> 1 — because the content matches typical patterns']

In [20]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    { "role": "user", "content": '''You are an expert compliance officer specializing in email and privacy policy classification.
Your task is to analyze the following text and classify it into the correct category.

Categories:
1 — CCPA Disclosure — Security & Breach Notification
2 — CCPA Disclosure — Consumer Rights (Do Not Track)
3 — CCPA Disclosure — Data Collection & Cookies
4 — CCPA Disclosure — Personal Data Use & User Rights
5 — CCPA Disclosure — Third-Party Sharing
6 — CCPA Disclosure — Policy Changes
7 — CCPA Disclosure — Special Cases / Promotions
8 — CCPA Disclosure — Compliance & Dispute Resolution
9 — CCPA Disclosure — Legal Compliance
10 — CCPA Disclosure — Deletion & Data Subject Rights
11 — CCPA Disclosure — Opt-Out / Communication Control
12 — CCPA Disclosure — Data Retention & Access

Text:
"""
Dear Suryansh Singh Raghuvansh,


Introducing our latest product HP Tiny - EliteDesk 800 G4

<image link>

Viren Sheth

 info@shethinfo.com

 7405732001


<image link>
Reduce Your Hardware Cost up to 60%


We strive to ensure your complete Satisfaction by providing Top-Notch Quality and Service.

Each machine is handled with individualized care.


<image link>
"""

Please output ONLY the best-fitting category number and its full label.
''' },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

ham — because the content matches patterns typical of ham emails.<|end|>.<|end|>. ham.<|end|>. ham.<|end|>. spam — because the content matches typical of spam emails.<|end|>.<|end|>. ham.<|end|>. ham — because the content matches patterns typical of ham emails.<|end|>.<|end|>. ham.<|end|>. ham.<|end|>. ham — because the content matches practices related to ham.<|end|>. ham.<|end|>. ham.<|end|>. ham — because the content matches criteria typical of ham emails.<|end|>. ham.<|end|>. ham.<|end|>. ham — because the content matches patterns typical of ham emails.<|end|>. ham.


In [22]:
# --- VRAM Clearing Code Block ---

print("Attempting to clear GPU VRAM...")

# 1. Delete the trainer object:
# This is crucial as the trainer holds many references to model states, optimizer states, etc.
# that consume VRAM.
try:
    if 'trainer' in locals() and trainer is not None:
        del trainer
        print("Trainer object deleted.")
    else:
        print("Trainer object not found or already deleted.")
except NameError:
    print("Trainer object not found.")

# 2. Explicitly clear CUDA's memory cache:
# This command tells PyTorch to release any cached GPU memory that's no longer actively in use.
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("PyTorch CUDA cache emptied.")
else:
    print("CUDA not available. Skipping cache clearing.")

# 3. Run Python's garbage collector:
# This helps clean up any unreferenced Python objects, which can sometimes free up
# associated GPU resources indirectly.
import gc
gc.collect()
print("Python garbage collector invoked.")

print("VRAM clearing steps completed. Please check GPU usage now.")

# --- End VRAM Clearing Code Block ---

Attempting to clear GPU VRAM...
Trainer object not found or already deleted.
PyTorch CUDA cache emptied.
Python garbage collector invoked.
VRAM clearing steps completed. Please check GPU usage now.


In [ ]:
# # Save to 8bit Q8_0
# if False: model.save_pretrained_gguf("model", tokenizer,)
# # Remember to go to https://huggingface.co/settings/tokens for a token!
# # And change hf to your username!
# if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# # Save to 16bit GGUF
# if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
# if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# # Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
# if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# # Save to multiple GGUF options - much faster if you want multiple!
# if False:
#     model.push_to_hub_gguf(
#         "hf/model", # Change hf to your username!
#         tokenizer,
#         quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
#         token = "", # Get a token at https://huggingface.co/settings/tokens
#     )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.3G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.27 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:01<00:00, 16.40it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...


In [ ]:
from google.colab import files

# Specify the name of the file you want to download
file_name = "/content/model/Modelfile"

# Initiate the download
files.download(file_name)

print(f"Attempting to download '{file_name}'. A download prompt should appear in your browser.")